In [1]:
import numpy as np
import pandas as pd
import pickle
from IPython.core.display import HTML
import scipy.stats as stats
import statsmodels.stats.weightstats as wstats

<div class="span5 alert alert-info">
<h2>Note book details</h2>

<p> This notebook is for <b>data story</b> for <b>kickstarter success prediction</b> project.</p>

<p> Notes.</p>
<ol>
<li>  The file will statistical analysis to establish whether one field has a significant impact on the another</li> 
<li>  The file will address the explore some hypothesis on the data</li> 
<li>What are the null and alternate hypotheses?</li>
<li>Compute margin of error, confidence interval, and p-value.</li>
<li>Write a story describing the statistical significance in the context or the original problem.</li>
<li> Explore following data and hypothesis : 
    <ul>
     <li>Exploration 1 :No of backer for fail and successful projects</li>
    </ul>
</li>        
</ol>
</div>

In [2]:
pd.set_option('display.max_colwidth', -1)
color_success='#00ff66'#kickstarter green#'#2BDE73'
color_fail='#FF192F'#'#FFC7CE'#'#FFC7CE'#'#FF0000' # red
stateSuccess='Successful'
stateFail='Failed'
# the statistical significance value is 5%
siginificane_level =0.005
# for 95% the critcal z value is 1.96 ( based on z table)
critical_value =1.96
noofsample=100

In [3]:
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }'
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='DataStory_Jan2017_Jan2018.sa'
df_final_months = pickle.load( open( pickle_file, "rb" ) )
#df_final_months.info()
#df_final_months.head()

In [4]:
# filter out suspened, cancleed and live projects
df_fail_success= df_final_months[(df_final_months.state==stateSuccess) | (df_final_months.state==stateFail)]
_=df_fail_success.set_index('project_id')
#df_fail_success.head()

##  Analysis start

<div class="span5 alert alert-info">
<h4>
Exploration 1 :No of backer for fail and successful projects</h4>
</div>

In [5]:
analyzecol='backers_count'
dfHypo=df_fail_success.loc[:,['state',analyzecol,'usd_goal']]

#### Get the dataframe

In [6]:
#dfHypo_success=GetCLTDataframe(dfHypo[dfHypo.state==stateSuccess],analyzecol,noofsample)
#dfHypo_fail=GetCLTDataframe(dfHypo[dfHypo.state==stateFail],analyzecol,noofsample)

dfHypo_success=df_fail_success[dfHypo.state==stateSuccess].loc[:,['state',analyzecol]]
dfHypo_fail=df_fail_success[dfHypo.state==stateFail].loc[:,['state',analyzecol]]

In [7]:
dfHypo_success.backers_count.describe()

count    74847.000000 
mean     291.705680   
std      1844.760094  
min      1.000000     
25%      34.000000    
50%      72.000000    
75%      173.000000   
max      219382.000000
Name: backers_count, dtype: float64

In [8]:
dfHypo_fail.backers_count.describe()

count    82896.000000
mean     15.552765   
std      82.981983   
min      0.000000    
25%      1.000000    
50%      3.000000    
75%      10.000000   
max      6550.000000 
Name: backers_count, dtype: float64

In [9]:
mean_success_backers_count=dfHypo_success.backers_count.mean()
mean_fail_backers_count=dfHypo_fail.backers_count.mean()
mean_diff_backers_count=mean_success_backers_count-mean_fail_backers_count
mean_diff_backers_count


276.1529146761072

<div class="span5 alert alert-info" style="background-color:#ffff66; color:black">

There is difference between the mean of backer_counts between successful and failed project. We can further analyze this  hypothesis. <b>Answer :</b> The hypothesis are as follows
    <br/>
    <b> Null Hypothesis : </b> There is <b>no</b> difference in no of backer in successful or failed project. Which <i>means for no of backers for successful - means for no of backers for failed</i> equals <b>Zero</b> .
    <br/>
    <b> Alternate Hypothesis :  </b> There is <b>siginifcant</b> difference in no of backer in successful or failed project. Which <i>means for no of backers for successful - means for no of backers for failed</i> not equals <b>Zero</b> .
</div>

#### Calculate Z scorw and p score for this 

In [10]:
zstat, pvalue = wstats.ztest(dfHypo_success[analyzecol], dfHypo_fail[analyzecol], alternative='two-sided',
                    value=0, usevar='pooled', ddof=1.0)
print ('The z stat is {} and p value is {}'.format(zstat,pvalue))

The z stat is 43.05172748164478 and p value is 0.0


In [11]:
zstat, pvalue = wstats.ztest(dfHypo_success[analyzecol], dfHypo_fail[analyzecol], alternative='larger',#'smaller',
                    value=0, usevar='pooled', ddof=1.0)
print ('The z stat is {} and p value is {}'.format(zstat,pvalue))

The z stat is 43.05172748164478 and p value is 0.0


####  Calculate T score and p value to test same hypothesis

In [12]:
# T-score
tstat, p_from_t = stats.ttest_ind(dfHypo_success[analyzecol], dfHypo_fail[analyzecol], equal_var=False)
print('t-statistic: ', tstat)
print('p-value: ', p_from_t)

t-statistic:  40.9166979163
p-value:  0.0


In [13]:
if (p_from_t<siginificane_level):
    print ('The calulcated p value is lower then 0.05 % significant value will allow to reject the null hypothesis')
else:
    print('Null hypotheses is true')

The calulcated p value is lower then 0.05 % significant value will allow to reject the null hypothesis


<div class="span5 alert alert-info analysis" >
<h4>
Conclusion : Exploration 1 :No of backer for fail and successful projects</h4>
    <ul>
     <li> There is enough evidence to reject Null hypothesis </li> 
     <li> The conculsion is that no of backer for failed and successful  project vaires </li> 
    </ul>
</div>